In [1]:
# Célula 1: Carregar Bibliotecas e Dados

import pandas as pd

# Ajuste o nome do arquivo, separador (sep), decimal e adicione a codificação!
caminho_arquivo = 'STP-20251124190950218.csv' 
df = pd.read_csv(caminho_arquivo, sep=';', decimal=',', encoding='latin-1') 

# ----------------------------------------------------
# Inspecionar o DataFrame

print("Nomes das colunas:")
print(df.columns)
print("-" * 30)
print("Primeiras 5 linhas (Formato dos dados):")
print(df.head())

Nomes das colunas:
Index(['Data', '7465 - Índice de Preços ao Consumidor (IPC-Fipe) - Alimentação - Var. % mensal'], dtype='object')
------------------------------
Primeiras 5 linhas (Formato dos dados):
      Data  \
0  01/1997   
1  02/1997   
2  03/1997   
3  04/1997   
4  05/1997   

  7465 - Índice de Preços ao Consumidor (IPC-Fipe) - Alimentação - Var. % mensal  
0                                               1,51                              
1                                               0,76                              
2                                               1,59                              
3                                              -0,37                              
4                                              -1,37                              


In [2]:
## Célula de Tratamento de Dados (Para Séries Temporais)

# O nome da coluna longa que identificamos
NOME_COLUNA_LONGA = '7465 - Índice de Preços ao Consumidor (IPC-Fipe) - Alimentação - Var. % mensal'
NOME_COLUNA_ALVO = 'Var_IPC_Alimentacao_Mensal'

# Passo 1A: Renomear, Corrigir e Converter a Variável Alvo para PODER LIMPAR

# Renomear (Para facilitar o acesso)
df.rename(columns={NOME_COLUNA_LONGA: NOME_COLUNA_ALVO}, inplace=True)

# Corrigir o formato: Substituir a vírgula (,) por ponto (.)
df[NOME_COLUNA_ALVO] = df[NOME_COLUNA_ALVO].astype(str).str.replace(',', '.', regex=False)

# Passo 1B: Limpar dados ruins (como a linha "Fonte")
# Identificamos e removemos linhas onde a coluna ALVO não é um número válido (texto, notas de rodapé, etc.)
valores_numericos = pd.to_numeric(df[NOME_COLUNA_ALVO], errors='coerce') 
df_limpo = df[valores_numericos.notna()].copy() 

# ----------------------------------------------------

# Passo 2: Agora que o DataFrame está limpo, podemos tratar a data.
df_limpo['Data'] = pd.to_datetime(df_limpo['Data'], format='%m/%Y')
df_limpo.set_index('Data', inplace=True)
df_ts = df_limpo.sort_index()

# Passo 3: Conversão final para float e remoção de NaNs remanescentes
df_ts[NOME_COLUNA_ALVO] = pd.to_numeric(df_ts[NOME_COLUNA_ALVO]) # Conversão final
df_final = df_ts[[NOME_COLUNA_ALVO]].dropna()

# Passo 4: Salvar o Arquivo Limpo
df_final.to_csv('dados_inflacao_corrigidos.csv', index=True, sep=',', decimal='.')

print("Tratamento de Dados Finalizado.")
print("Série Temporal (IPC Alimentação) pronta para modelagem:")
print(df_final.head())
print(df_final.info())

Tratamento de Dados Finalizado.
Série Temporal (IPC Alimentação) pronta para modelagem:
            Var_IPC_Alimentacao_Mensal
Data                                  
1997-01-01                        1.51
1997-02-01                        0.76
1997-03-01                        1.59
1997-04-01                       -0.37
1997-05-01                       -1.37
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 346 entries, 1997-01-01 to 2025-10-01
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Var_IPC_Alimentacao_Mensal  346 non-null    float64
dtypes: float64(1)
memory usage: 5.4 KB
None


In [3]:
# 1. Renomear a coluna longa para um nome simples
NOME_COLUNA_ALVO = 'Var_IPC_Alimentacao_Mensal'
df_ts.rename(columns={df_ts.columns[0]: NOME_COLUNA_ALVO}, inplace=True)

# 2. Corrigir o formato: Substituir a vírgula (,) por ponto (.) e converter para float
# Se a leitura inicial com 'decimal=,' não converteu, fazemos explicitamente agora.
df_ts[NOME_COLUNA_ALVO] = df_ts[NOME_COLUNA_ALVO].astype(str).str.replace(',', '.', regex=False)
df_ts[NOME_COLUNA_ALVO] = pd.to_numeric(df_ts[NOME_COLUNA_ALVO])

# Selecionar apenas a série temporal final e remover NaNs
df_final = df_ts[[NOME_COLUNA_ALVO]].dropna()

In [3]:
# Salvar o DataFrame final pronto para modelagem
# Usando separador (sep) e decimal padrão para ML (vírgula e ponto)
df_final.to_csv('dados_inflacao_corrigidos.csv', index=True, sep=',', decimal='.')

print("Tratamento de Dados Finalizado.")
print("Série Temporal (IPC Alimentação) pronta para modelagem:")
print(df_final.head())
print(df_final.info())

Tratamento de Dados Finalizado.
Série Temporal (IPC Alimentação) pronta para modelagem:
            Var_IPC_Alimentacao_Mensal
Data                                  
1997-01-01                        1.51
1997-02-01                        0.76
1997-03-01                        1.59
1997-04-01                       -0.37
1997-05-01                       -1.37
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 346 entries, 1997-01-01 to 2025-10-01
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Var_IPC_Alimentacao_Mensal  346 non-null    float64
dtypes: float64(1)
memory usage: 5.4 KB
None


## 3. Formulação do Problema e Desenvolvimento do Deep Learning (Etapas 3 e 4)

O problema formulado é de **Previsão de Séries Temporais**. Utilizou-se a arquitetura **Long Short-Term Memory (LSTM)**, especializada na captura de dependências temporais de longo prazo.

O desenvolvimento completo do pipeline de preparação de sequências, definição da arquitetura, treinamento e avaliação do modelo encontra-se no script de produção:

[Link para o Código de Modelagem: `Modelagem_LSTM.py`](./Modelagem_LSTM.py)

### Principais Resultados do Modelo

* **RMSE de Treino:** 0.8949
* **RMSE de Teste:** 0.7674
* **Gráfico de Predição:** (Cole aqui a imagem do gráfico que você gerou na Célula 5).